# Бейзлайн решение

В рамках олимпиады вам предстоит решить задачу по адаптация скорости LDPC-кодов для постобработки в квантовой криптографии.

В данном Jupyter ноутбуке представлено бейзлайн решение, которое позволяет получить файл предсказания в нужном для проверяющей системы формате. Он тестировался с библиотеками из ```requirements.txt``` и версией ```Python 3.12.11```.

#### Желаем удачи!

# Препроцессинг

In [1]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("frames_errors.csv", header=None)
df.columns = [
    "block_id",
    "frame_idx",
    "E_mu_Z",
    "E_mu_phys_est",
    "E_mu_X",
    "E_nu1_X",
    "E_nu2_X",
    "E_nu1_Z",
    "E_nu2_Z",
    "N_mu_X",
    "M_mu_XX",
    "M_mu_XZ",
    "M_mu_X",
    "N_mu_Z",
    "M_mu_ZZ",
    "M_mu_Z",
    "N_nu1_X",
    "M_nu1_XX",
    "M_nu1_XZ",
    "M_nu1_X",
    "N_nu1_Z",
    "M_nu1_ZZ",
    "M_nu1_Z",
    "N_nu2_X",
    "M_nu2_XX",
    "M_nu2_XZ",
    "M_nu2_X",
    "N_nu2_Z",
    "M_nu2_ZZ",
    "M_nu2_Z",
    "nTot",
    "bayesImVoltage",
    "opticalPower",
    "polarizerVoltages[0]",
    "polarizerVoltages[1]",
    "polarizerVoltages[2]",
    "polarizerVoltages[3]",
    "temp_1",
    "biasVoltage_1",
    "temp_2",
    "biasVoltage_2",
    "synErr",
    "N_EC_rounds",
    "maintenance_flag",
    "estimator_name",
    "f_EC",
    "E_mu_Z_est",
    "R",
    "s",
    "p",
]

df_base = df.drop(
    [
        "E_mu_phys_est",
        "f_EC",
    ],
    axis=1,
)
print(f"Количество пропусков: {df.isna().sum().sum()}")

Количество пропусков: 579


In [3]:
df = df_base.copy()

In [4]:
df = df.rename(
    columns={
        "block_id": "id",
        "E_mu_Z": "value",
        "frame_idx": "date",
    }
)

# Смотрим на длину временных рядов по количеству фреймов
timestamp_counts = df.groupby("id")["date"].nunique()
print("Количество фреймов/Количество рядов")
print(timestamp_counts.value_counts())

df_for_ts = df[["id", "value", "date"]].dropna(subset=["value"], how="any")

Количество фреймов/Количество рядов
date
399    569
400    251
398      2
390      1
Name: count, dtype: int64


In [5]:
df_for_ts = df_for_ts.set_index(["id", "date"]).unstack().ffill().stack().reset_index()
timestamp_counts = df_for_ts.groupby("id")["date"].nunique()
print("Количество фреймов/Количество рядов")
print(timestamp_counts.value_counts())

Количество фреймов/Количество рядов
date
400    815
399      8
Name: count, dtype: int64


In [6]:
df_for_ts = df_for_ts.groupby("id").filter(lambda x: len(x) == 400)
print("Оставшиеся сегменты:", df_for_ts["id"].nunique())

Оставшиеся сегменты: 815


# DLinear

`DLinear` — это простая и быстрая модель, которая выделяет тренд на основе AveragePooling, а затем на компонентах тренда и остатков применяет nn.Linear и собирает всё обратно. Ознакомиться с моделью можно в статье https://arxiv.org/abs/2205.13504

In [7]:
import logging

logger = logging.getLogger(__name__)
import sys

c_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(c_handler)
logging.basicConfig(level=logging.INFO, force=True)

import random
import warnings


import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from torch.nn import Module

from tsururu.dataset import Pipeline, TSDataset
from tsururu.model_training.trainer import DLTrainer
from tsururu.model_training.validator import HoldOutValidator
from tsururu.strategies import RecursiveStrategy

warnings.filterwarnings("ignore")

In [8]:
class moving_avg(Module):
    """Блок скользящего среднего для выделения тренда временного ряда.

    Аргументы:
        kernel_size: размер окна свёртки (ядра).
        stride: шаг скользящего среднего.

    """

    def __init__(self, kernel_size: int, stride: int):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x: "torch.Tensor") -> "torch.Tensor":
        """Прямой проход для вычисления скользящего среднего.

        Аргументы:
            x: входной тензор.

        Возвращает:
            тензор после применения скользящего среднего.

        """
        # добавляем паддинг (повторяем крайние значения) с обеих сторон временного ряда
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)

        # применяем скользящее среднее по временной оси
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)

        return x


class series_decomp(Module):
    """Блок декомпозиции временного ряда.

    Аргументы:
        kernel_size: размер окна для скользящего среднего.

    """

    def __init__(self, kernel_size: int):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x: "torch.Tensor") -> tuple["torch.Tensor", "torch.Tensor"]:
        """Прямой проход для декомпозиции ряда на тренд и остаток.

        Аргументы:
            x: входной тензор.

        Возвращает:
            кортеж тензоров (остаток, тренд).

        """
        moving_mean = self.moving_avg(x)
        res = x - moving_mean

        return res, moving_mean


class DLinear_NN(Module):
    def __init__(self, features_groups, pred_len, seq_len, moving_avg=25, **kwargs):
        super().__init__()

        # Защита от типовых «обёрток»
        def _to_int(x):
            if isinstance(x, int):
                return x
            if isinstance(x, dict) and "value" in x:
                return int(x["value"])
            try:
                return int(x)
            except Exception:
                raise TypeError(f"Expected int-like, got {type(x)}: {x}")

        # Если вдруг пришли ещё и именованные — заберём их, чтобы не мешали
        seq_len = _to_int(kwargs.pop("seq_len", seq_len))
        pred_len = _to_int(kwargs.pop("pred_len", pred_len))
        moving_avg = int(kwargs.pop("moving_avg", moving_avg))

        self.seq_len = seq_len
        self.pred_len = pred_len

        self.decompsition = series_decomp(moving_avg)
        self.Linear_Seasonal = nn.Linear(self.seq_len, self.pred_len)
        self.Linear_Trend = nn.Linear(self.seq_len, self.pred_len)

        self.Linear_Seasonal.weight = nn.Parameter(
            (1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len])
        )
        self.Linear_Trend.weight = nn.Parameter(
            (1 / self.seq_len) * torch.ones([self.pred_len, self.seq_len])
        )

    def forward(self, x: "torch.Tensor") -> "torch.Tensor":
        """Прямой проход модели.

        Аргументы:
            x: входной тензор формы (batch_size, seq_len, num_features).

        Возвращает:
            выходной тензор формы (batch_size, pred_len, num_features).

        """
        # Декомпозиция временного ряда на тренд и остаток (сезонность)
        seasonal_init, trend_init = self.decompsition(x)

        # Транспонируем тензоры в формат (batch_size, num_features, seq_len)
        seasonal_init, trend_init = seasonal_init.permute(0, 2, 1), trend_init.permute(
            0, 2, 1
        )

        # Применяем линейные слои к тренду и остаткам
        seasonal_output = self.Linear_Seasonal(seasonal_init)
        trend_output = self.Linear_Trend(trend_init)

        # Складываем результаты линейных слоёв
        x = seasonal_output + trend_output

        # Транспонируем обратно в формат (batch_size, seq_len, num_features)
        x = x.permute(0, 2, 1)

        return x[:, -self.pred_len :, :]

In [9]:
# Будем предсказывать 8 чисел вперед с окном 160

HORIZON = 8
HISTORY = 160

In [10]:
train_df = []
val_df = []
test_df = []
test_targets = []
for current_id in df_for_ts["id"].unique():
    current_df = df_for_ts[df_for_ts["id"] == current_id]
    train_df.append(current_df.iloc[: -2 * HORIZON])
    val_df.append(current_df.iloc[-2 * HORIZON - HISTORY : -HORIZON])
    test_df.append(current_df.iloc[-HORIZON - HISTORY : -HORIZON])
    test_targets.append(current_df.iloc[-HORIZON:])
train_df = pd.concat(train_df)
val_df = pd.concat(val_df)
test_df = pd.concat(test_df)
test_targets = pd.concat(test_targets)


print(f"Форма обучающего набора: {train_df.shape}")
print(f"Форма валидационного набора: {val_df.shape}")
print(f"Форма тестового набора: {test_df.shape}")
print(f"Форма целевых значений теста: {test_targets.shape}")

print(f"Количество рядов в обучающем наборе: {train_df['id'].nunique()}")
print(f"Количество рядов в валидационном наборе: {val_df['id'].nunique()}")
print(f"Количество рядов в тестовом наборе: {test_df['id'].nunique()}")
print(f"Количество рядов в целевых значениях теста: {test_targets['id'].nunique()}")

Форма обучающего набора: (312960, 3)
Форма валидационного набора: (136920, 3)
Форма тестового набора: (130400, 3)
Форма целевых значений теста: (6520, 3)
Количество рядов в обучающем наборе: 815
Количество рядов в валидационном наборе: 815
Количество рядов в тестовом наборе: 815
Количество рядов в целевых значениях теста: 815


In [11]:
# Устанавливаем базовую дату (первый день)
# Это необходимо для корректной работы библиотеки tsururu и не влияет на суть задачи

base_date = pd.to_datetime("2000-01-01")


def convert_dates(series):
    return base_date + pd.to_timedelta(series.astype(int) - 1, unit="D")


# Применяем ко всем DataFrame

train_df["date"] = convert_dates(train_df["date"])
val_df["date"] = convert_dates(val_df["date"])
test_df["date"] = convert_dates(test_df["date"])
test_targets["date"] = convert_dates(test_targets["date"])

In [12]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [13]:
seed_everything()
dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continuous",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    },
}

train_dataset = TSDataset(
    data=train_df,
    columns_params=dataset_params,
    print_freq_period_info=True,
)
val_dataset = TSDataset(
    data=val_df,
    columns_params=dataset_params,
    print_freq_period_info=False,
)
test_dataset = TSDataset(
    data=test_df,
    columns_params=dataset_params,
    print_freq_period_info=False,
)

INFO:tsururu.dataset.dataset:freq: Day; period: 1


In [14]:
pipeline_params = {
    "target": {
        "columns": ["value"],
        "features": {
            "DifferenceNormalizer": {
                "regime": "delta",
                "transform_target": True,
                "transform_features": True,
            },
            "MissingValuesImputer": {  # После DifferenceNormalizer у нас неизбежно появляются NaN в данных (в первом значении каждого сегмента)
                "constant_value": 0,  # Заполним нулями
                "transform_target": True,
                "transform_features": True,
            },
            "StandardScalerTransformer": {  # И выровним значения рядов прежде чем подавать в DL модель
                "transform_target": True,
                "transform_features": True,
                "agg_by_id": True,
            },
            "LagTransformer": {"lags": HISTORY},
        },
    }
}

In [15]:
def choose_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using GPU")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS")
    else:
        device = torch.device("cpu")
        print("Using CPU")
    return device

In [16]:
DEVICE = choose_device()

Using GPU


In [ ]:
# Настроим обучение

pipeline = Pipeline.from_dict(pipeline_params, multivariate=False)

validation = HoldOutValidator
validation_params = {"validation_data": val_dataset}

trainer_params = {
    "device": DEVICE,
    "num_workers": 4,
    "best_by_metric": True,
    "save_to_dir": False,
    "batch_size": 256,
    "n_epochs": 5,
    "early_stopping_patience": 2,
}


trainer = DLTrainer(
    model=DLinear_NN,
    model_params={"moving_avg": 25},
    validator=validation,
    validation_params=validation_params,
    **trainer_params,
)


strategy = RecursiveStrategy(
    horizon=HORIZON,
    model_horizon=4,
    history=HISTORY,
    pipeline=pipeline,
    trainer=trainer,
)

In [18]:
# Запустим обучение

fit_time, metrics = strategy.fit(train_dataset)

INFO:tsururu.model_training.trainer:length of train dataset: 180115
INFO:tsururu.model_training.trainer:length of val dataset: 4075
INFO:tsururu.model_training.trainer:Epoch 1/5, cost time: 96.20s
INFO:tsururu.model_training.trainer:train loss: 0.7057
INFO:tsururu.model_training.trainer:Validation, Loss: 0.6650, Metric: -0.6650
INFO:tsururu.model_training.trainer:val loss: 0.6650, val metric: -0.6650
INFO:tsururu.model_training.trainer:Epoch 2/5, cost time: 93.20s
INFO:tsururu.model_training.trainer:train loss: 0.6827
INFO:tsururu.model_training.trainer:Validation, Loss: 0.6613, Metric: -0.6613
INFO:tsururu.model_training.trainer:val loss: 0.6613, val metric: -0.6613
INFO:tsururu.model_training.trainer:Epoch 3/5, cost time: 90.28s
INFO:tsururu.model_training.trainer:train loss: 0.6813
INFO:tsururu.model_training.trainer:Validation, Loss: 0.6624, Metric: -0.6624
INFO:tsururu.model_training.trainer:val loss: 0.6624, val metric: -0.6624
INFO:tsururu.model_training.torch_based.callbacks:Ea

In [19]:
# Сохраним модель для предоставления весов жюри

import pickle

model_filename = "dlinear_strategy.pkl"
with open(model_filename, "wb") as f:
    pickle.dump(strategy, f)

In [20]:
# Для предсказания загрузим уже обученную модель

with open(model_filename, "rb") as f:
    loaded_strategy = pickle.load(f)

In [21]:
forecast_time, current_pred = loaded_strategy.predict(test_dataset)

INFO:tsururu.dataset.dataset:freq: Day; period: 1
INFO:tsururu.model_training.trainer:length of test dataset: 815
INFO:tsururu.model_training.trainer:length of test dataset: 815


In [22]:
current_pred

,id,date,value
0,17612792,2001-01-26,0.018134
1,17612792,2001-01-27,0.017268
2,17612792,2001-01-28,0.018611
3,17612792,2001-01-29,0.020405
4,17612792,2001-01-30,0.019927
...,...,...,...
6515,2146878613,2001-01-29,0.023127
6516,2146878613,2001-01-30,0.02234
6517,2146878613,2001-01-31,0.02058
6518,2146878613,2001-02-01,0.018715


In [23]:
current_pred = current_pred.sort_values(["id", "date"]).reset_index(drop=True)

ids = current_pred["id"].unique().tolist()
n_ids = len(ids)

In [24]:
# Нам нужно вернуть 2000 точек значений
TOTAL = 2000
base = TOTAL // n_ids  # базовое число точек на id
rem = TOTAL % n_ids  # первым rem id дадим на 1 точку больше

if base == 0:
    # Случай, если рядов слишком много (n_ids > 2000): берём по 1 точке для первых 2000 id
    selected_ids = ids[:TOTAL]
    compressed_values = []
    for i in selected_ids:
        arr = current_pred.loc[current_pred["id"] == i, "value"].to_numpy()
        # берём, например, последнее значение горизонта
        compressed_values.append(float(arr[-1]))
else:
    # Обычный случай (~815 рядов): base=2, rem=2000-2*815=370 => 370 рядов дадут 3 точки, остальные 2
    compressed_values = []
    for idx, i in enumerate(ids):
        k = base + (1 if idx < rem else 0)  # целевых точек для этого id
        arr = current_pred.loc[current_pred["id"] == i, "value"].to_numpy()

        # Защита: если горизонт < k (не должно быть), просто повторим последние
        if len(arr) < k:
            arr = np.pad(arr, (0, k - len(arr)), mode="edge")

        # Режем на k ~равных кусков и усредняем каждый
        chunks = np.array_split(arr, k)
        means = [float(np.mean(c)) for c in chunks]
        compressed_values.extend(means)

# Получаем ровно 2000 значений в фиксированном порядке
target_df = pd.DataFrame({"value": compressed_values})
assert len(target_df) == 2000, f"Got {len(target_df)} instead of 2000"

In [25]:
from math import ceil

alpha = 0.33
f_ec = 1.15
R_range = [
    round(0.50 + 0.05 * x, 2) for x in range(9)
]  # 0.50..0.90 для соответствия условию задачи
n = 32000
d = 4800

In [26]:
def calculate_ema(prev_ema, current_value, alpha):
    if prev_ema is None:
        return current_value
    return alpha * current_value + (1 - alpha) * prev_ema


def h(x):
    if x > 0:
        return -x * np.log2(x) - (1 - x) * np.log2(1 - x)
    elif x == 0:
        return 0.0
    else:
        raise ValueError("Invalid x for binary entropy")


def select_code_rate(e_mu, f_ec, rates, frame_len, sp_count):
    r_candidate = 1 - h(e_mu) * f_ec
    R_res = 0.50
    s_n = sp_count
    p_n = 0
    for R in rates:
        p_n = int(
            ceil((1 - R) * frame_len - (1 - r_candidate) * (frame_len - sp_count))
        )
        s_n = int(sp_count - p_n)
        if p_n >= 0 and s_n >= 0:
            R_res = R
            return round(R_res, 2), s_n, p_n
    return round(R_res, 2), s_n, p_n

In [27]:
E_series = (
    pd.to_numeric(target_df.iloc[:, 0], errors="coerce").dropna().reset_index(drop=True)
)

prev_ema = None
rows = []
for E_mu_Z in E_series:
    ema_value = calculate_ema(prev_ema, float(E_mu_Z), alpha)
    prev_ema = ema_value
    R, s_n, p_n = select_code_rate(ema_value, f_ec, R_range, n, d)
    rows.append([f"{E_mu_Z:.16f}", R, s_n, p_n])  # 4 столбца: E, R, s_n, p_n

# Сохраним результат
pd.DataFrame(rows).to_csv("submission.csv", header=False, index=False)

*Увеличение количества признаков*

Количество пропусков: 579
Используется 16 признаков:
Оставшиеся сегменты: 815
Форма обучающего набора: (312960, 18)
Используется признаков: 16
freq: Day; period: 1
Using GPU
Обучение улучшенной NLinear модели с расширенными настройками...
length of train dataset: 221
length of val dataset: 5
Epoch 1/15, cost time: 48.37s
train loss: 1.9644


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [1]:
# =============================================================================
# БАЗОВОЕ РЕШЕНИЕ С УСИЛЕННОЙ МОДЕЛЬЮ (БЕЗ ДОПОЛНИТЕЛЬНЫХ ПРИЗНАКОВ)
# =============================================================================

import warnings
import pandas as pd

warnings.filterwarnings("ignore")

# Загрузка и подготовка данных (ТОЧНО КАК В БАЗОВОМ РЕШЕНИИ)
df = pd.read_csv("frames_errors.csv", header=None)
df.columns = [
    "block_id", "frame_idx", "E_mu_Z", "E_mu_phys_est", "E_mu_X", "E_nu1_X", "E_nu2_X", 
    "E_nu1_Z", "E_nu2_Z", "N_mu_X", "M_mu_XX", "M_mu_XZ", "M_mu_X", "N_mu_Z", "M_mu_ZZ", 
    "M_mu_Z", "N_nu1_X", "M_nu1_XX", "M_nu1_XZ", "M_nu1_X", "N_nu1_Z", "M_nu1_ZZ", 
    "M_nu1_Z", "N_nu2_X", "M_nu2_XX", "M_nu2_XZ", "M_nu2_X", "N_nu2_Z", "M_nu2_ZZ", 
    "M_nu2_Z", "nTot", "bayesImVoltage", "opticalPower", "polarizerVoltages[0]", 
    "polarizerVoltages[1]", "polarizerVoltages[2]", "polarizerVoltages[3]", "temp_1", 
    "biasVoltage_1", "temp_2", "biasVoltage_2", "synErr", "N_EC_rounds", "maintenance_flag", 
    "estimator_name", "f_EC", "E_mu_Z_est", "R", "s", "p"
]

df_base = df.drop(["E_mu_phys_est", "f_EC"], axis=1)
print(f"Количество пропусков: {df.isna().sum().sum()}")

df = df_base.copy()

df = df.rename(columns={"block_id": "id", "E_mu_Z": "value", "frame_idx": "date"})

# Смотрим на длину временных рядов по количеству фреймов
timestamp_counts = df.groupby("id")["date"].nunique()
print("Количество фреймов/Количество рядов")
print(timestamp_counts.value_counts())

df_for_ts = df[["id", "value", "date"]].dropna(subset=["value"], how="any")
df_for_ts = df_for_ts.set_index(["id", "date"]).unstack().ffill().stack().reset_index()
timestamp_counts = df_for_ts.groupby("id")["date"].nunique()
print("Количество фреймов/Количество рядов")
print(timestamp_counts.value_counts())
df_for_ts = df_for_ts.groupby("id").filter(lambda x: len(x) == 400)
print("Оставшиеся сегменты:", df_for_ts["id"].nunique())

import logging
import random
import warnings
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import Module
from tsururu.dataset import Pipeline, TSDataset
from tsururu.model_training.trainer import DLTrainer
from tsururu.model_training.validator import HoldOutValidator
from tsururu.strategies import RecursiveStrategy

warnings.filterwarnings("ignore")

# =============================================================================
# УСИЛЕННАЯ МОДЕЛЬ ВМЕСТО DLinear
# =============================================================================

class EnhancedTemporalModel(Module):
    """
    Усиленная модель для временных рядов с механизмами внимания
    Комбинация CNN + LSTM + Attention
    """
    
    def __init__(self, features_groups, pred_len, seq_len, **kwargs):
        super().__init__()
        
        # Параметры
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.hidden_dim = 128
        self.num_layers = 2
        
        # 1. CNN для извлечения локальных временных паттернов
        self.conv1d = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.AdaptiveAvgPool1d(seq_len // 2)  # уменьшаем размерность
        )
        
        # 2. LSTM для долгосрочных зависимостей
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
            dropout=0.2,
            bidirectional=True
        )
        
        # 3. Механизм внимания для важных временных точек
        self.attention = nn.Sequential(
            nn.Linear(self.hidden_dim * 2, 64),  # bidirectional -> *2
            nn.Tanh(),
            nn.Linear(64, 1),
            nn.Softmax(dim=1)
        )
        
        # 4. Полносвязные слои для прогнозирования
        self.fc = nn.Sequential(
            nn.Linear(self.hidden_dim * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, pred_len)
        )
        
        # Инициализация весов
        self._initialize_weights()
        
    def _initialize_weights(self):
        """Инициализация весов для лучшей сходимости"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.LSTM):
                for name, param in module.named_parameters():
                    if 'weight' in name:
                        nn.init.orthogonal_(param)
                    elif 'bias' in name:
                        nn.init.constant_(param, 0)
                        
    def forward(self, x):
        """
        x shape: (batch_size, seq_len, 1)
        """
        batch_size = x.shape[0]
        
        # 1. Применяем CNN
        x_cnn = x.permute(0, 2, 1)  # (batch_size, 1, seq_len)
        x_cnn = self.conv1d(x_cnn)   # (batch_size, 128, seq_len//2)
        x_cnn = x_cnn.permute(0, 2, 1)  # (batch_size, seq_len//2, 128)
        
        # 2. Применяем LSTM
        lstm_out, (h_n, c_n) = self.lstm(x_cnn)  # (batch_size, seq_len//2, hidden_dim*2)
        
        # 3. Применяем механизм внимания
        attention_weights = self.attention(lstm_out)  # (batch_size, seq_len//2, 1)
        context_vector = torch.sum(attention_weights * lstm_out, dim=1)  # (batch_size, hidden_dim*2)
        
        # 4. Прогнозирование
        output = self.fc(context_vector)  # (batch_size, pred_len)
        
        return output.unsqueeze(-1)  # (batch_size, pred_len, 1)

# =============================================================================
# ОСТАЛЬНОЙ КОД ТОЧНО КАК В БАЗОВОМ РЕШЕНИИ
# =============================================================================

# Будем предсказывать 8 чисел вперед с окном 160
HORIZON = 8
HISTORY = 160

train_df = []
val_df = []
test_df = []
test_targets = []
for current_id in df_for_ts["id"].unique():
    current_df = df_for_ts[df_for_ts["id"] == current_id]
    train_df.append(current_df.iloc[: -2 * HORIZON])
    val_df.append(current_df.iloc[-2 * HORIZON - HISTORY : -HORIZON])
    test_df.append(current_df.iloc[-HORIZON - HISTORY : -HORIZON])
    test_targets.append(current_df.iloc[-HORIZON:])
train_df = pd.concat(train_df)
val_df = pd.concat(val_df)
test_df = pd.concat(test_df)
test_targets = pd.concat(test_targets)

print(f"Форма обучающего набора: {train_df.shape}")
print(f"Форма валидационного набора: {val_df.shape}")
print(f"Форма тестового набора: {test_df.shape}")
print(f"Форма целевых значений теста: {test_targets.shape}")

print(f"Количество рядов в обучающем наборе: {train_df['id'].nunique()}")
print(f"Количество рядов в валидационном наборе: {val_df['id'].nunique()}")
print(f"Количество рядов в тестовом наборе: {test_df['id'].nunique()}")
print(f"Количество рядов в целевых значениях теста: {test_targets['id'].nunique()}")

# Устанавливаем базовую дату (первый день)
base_date = pd.to_datetime("2000-01-01")

def convert_dates(series):
    return base_date + pd.to_timedelta(series.astype(int) - 1, unit="D")

# Применяем ко всем DataFrame
train_df["date"] = convert_dates(train_df["date"])
val_df["date"] = convert_dates(val_df["date"])
test_df["date"] = convert_dates(test_df["date"])
test_targets["date"] = convert_dates(test_targets["date"])

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continuous",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    },
}

train_dataset = TSDataset(
    data=train_df,
    columns_params=dataset_params,
    print_freq_period_info=True,
)
val_dataset = TSDataset(
    data=val_df,
    columns_params=dataset_params,
    print_freq_period_info=False,
)
test_dataset = TSDataset(
    data=test_df,
    columns_params=dataset_params,
    print_freq_period_info=False,
)

pipeline_params = {
    "target": {
        "columns": ["value"],
        "features": {
            "DifferenceNormalizer": {
                "regime": "delta",
                "transform_target": True,
                "transform_features": True,
            },
            "MissingValuesImputer": {
                "constant_value": 0,
                "transform_target": True,
                "transform_features": True,
            },
            "StandardScalerTransformer": {
                "transform_target": True,
                "transform_features": True,
                "agg_by_id": True,
            },
            "LagTransformer": {"lags": HISTORY},
        },
    }
}

def choose_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using GPU")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS")
    else:
        device = torch.device("cpu")
        print("Using CPU")
    return device

DEVICE = choose_device()

# Настроим обучение С УСИЛЕННОЙ МОДЕЛЬЮ
pipeline = Pipeline.from_dict(pipeline_params, multivariate=False)

validation = HoldOutValidator
validation_params = {"validation_data": val_dataset}

# Параметры обучения (можно оставить как в базовом решении)
trainer_params = {
    "device": DEVICE,
    "num_workers": 4,
    "best_by_metric": True,
    "save_to_dir": False,
    "batch_size": 128,
    "n_epochs": 5,
    "early_stopping_patience": 2,
}

# ЕДИНСТВЕННОЕ ИЗМЕНЕНИЕ: используем усиленную модель вместо DLinear
trainer = DLTrainer(
    model=EnhancedTemporalModel,  # ← ЗДЕСЬ ЕДИНСТВЕННАЯ ЗАМЕНА
    model_params={},
    validator=validation,
    validation_params=validation_params,
    **trainer_params,
)

strategy = RecursiveStrategy(
    horizon=HORIZON,
    model_horizon=4,
    history=HISTORY,
    pipeline=pipeline,
    trainer=trainer,
)

# Запустим обучение
print("Обучение усиленной модели...")
fit_time, metrics = strategy.fit(train_dataset)
print(f"Обучение заняло {fit_time:.2f} секунд")
print(f"Метрики: {metrics}")

# Сохраним модель
import pickle
model_filename = "enhanced_temporal_model.pkl"
with open(model_filename, "wb") as f:
    pickle.dump(strategy, f)

# Для предсказания загрузим уже обученную модель
with open(model_filename, "rb") as f:
    loaded_strategy = pickle.load(f)

forecast_time, current_pred = loaded_strategy.predict(test_dataset)

current_pred = current_pred.sort_values(["id", "date"]).reset_index(drop=True)

ids = current_pred["id"].unique().tolist()
n_ids = len(ids)

# Нам нужно вернуть 2000 точек значений
TOTAL = 2000
base = TOTAL // n_ids
rem = TOTAL % n_ids

if base == 0:
    selected_ids = ids[:TOTAL]
    compressed_values = []
    for i in selected_ids:
        arr = current_pred.loc[current_pred["id"] == i, "value"].to_numpy()
        compressed_values.append(float(arr[-1]))
else:
    compressed_values = []
    for idx, i in enumerate(ids):
        k = base + (1 if idx < rem else 0)
        arr = current_pred.loc[current_pred["id"] == i, "value"].to_numpy()

        if len(arr) < k:
            arr = np.pad(arr, (0, k - len(arr)), mode="edge")

        chunks = np.array_split(arr, k)
        means = [float(np.mean(c)) for c in chunks]
        compressed_values.extend(means)

# Получаем ровно 2000 значений
target_df = pd.DataFrame({"value": compressed_values})
assert len(target_df) == 2000, f"Got {len(target_df)} instead of 2000"

from math import ceil

alpha = 0.33
f_ec = 1.15
R_range = [round(0.50 + 0.05 * x, 2) for x in range(9)]
n = 32000
d = 4800

def calculate_ema(prev_ema, current_value, alpha):
    if prev_ema is None:
        return current_value
    return alpha * current_value + (1 - alpha) * prev_ema

def h(x):
    if x > 0:
        return -x * np.log2(x) - (1 - x) * np.log2(1 - x)
    elif x == 0:
        return 0.0
    else:
        raise ValueError("Invalid x for binary entropy")

def select_code_rate(e_mu, f_ec, rates, frame_len, sp_count):
    r_candidate = 1 - h(e_mu) * f_ec
    R_res = 0.50
    s_n = sp_count
    p_n = 0
    for R in rates:
        p_n = int(ceil((1 - R) * frame_len - (1 - r_candidate) * (frame_len - sp_count)))
        s_n = int(sp_count - p_n)
        if p_n >= 0 and s_n >= 0:
            R_res = R
            return round(R_res, 2), s_n, p_n
    return round(R_res, 2), s_n, p_n

E_series = (
    pd.to_numeric(target_df.iloc[:, 0], errors="coerce").dropna().reset_index(drop=True)
)

prev_ema = None
rows = []
for E_mu_Z in E_series:
    ema_value = calculate_ema(prev_ema, float(E_mu_Z), alpha)
    prev_ema = ema_value
    R, s_n, p_n = select_code_rate(ema_value, f_ec, R_range, n, d)
    rows.append([f"{E_mu_Z:.16f}", R, s_n, p_n])

# Сохраним результат
pd.DataFrame(rows).to_csv("submission_enhanced_model.csv", header=False, index=False)

print("Усиленная модель обучена и submission создан!")
print("Использована та же логика, что и в базовом решении, только модель заменена на EnhancedTemporalModel")

Количество пропусков: 579
Количество фреймов/Количество рядов
date
399    569
400    251
398      2
390      1
Name: count, dtype: int64
Количество фреймов/Количество рядов
date
400    815
399      8
Name: count, dtype: int64
Оставшиеся сегменты: 815
Форма обучающего набора: (312960, 3)
Форма валидационного набора: (136920, 3)
Форма тестового набора: (130400, 3)
Форма целевых значений теста: (6520, 3)
Количество рядов в обучающем наборе: 815
Количество рядов в валидационном наборе: 815
Количество рядов в тестовом наборе: 815
Количество рядов в целевых значениях теста: 815
freq: Day; period: 1
Using GPU
Обучение усиленной модели...
length of train dataset: 180115
length of val dataset: 4075
Epoch 1/5, cost time: 99.14s
train loss: 0.6834
Validation, Loss: 0.6370, Metric: -0.6370
val loss: 0.6370, val metric: -0.6370
Epoch 2/5, cost time: 97.77s
train loss: 0.5926
Validation, Loss: 0.5288, Metric: -0.5288
val loss: 0.5288, val metric: -0.5288
Epoch 3/5, cost time: 98.60s
train loss: 0.54